In [1]:
import $file.hw3stdlib
import hw3stdlib._

import $file.$        

import hw3stdlib._

# Homework 3

In this assignment we will develop our first interpreter. But before this we will need to define the Integers and the arithmetic operations for them.

# $\mathbb{Z}$: The Integers

For the arithmetic interpreter we will write later on in the homework we will need arithmetic operations for the integers. These will obviously correspond very closely to the operations defined on the Natural Numbers. But now we will need to account for negative numbers in a proper way. 

Start by defining the Algebraic Datatype(sealed trait with case classes) for the Integers. The encoded type should be:

$$
\begin{align}
    \mathbb{Z} ::=&\ \text{Positive}\ \mathbb{N}\\
    \mid&\ \text{Negative}\ \mathbb{N}
\end{align}
$$

We will use this encoding because it will allow us to use some of our definitions for the Natural Numbers to define the operations on the Integers. This is the core strategy of functional programming: using smaller functions to build up larger ones. This is true even across types. We want to always minimize the ammount of code we have be reusing what we have already constructed, whenever possible. 

## Defining $\mathbb{Z}$ (5 points)

Define the ADT for Integers below: 

In [5]:
sealed trait Integer
case class Positive(x: Nat) extends Integer
case class Negative(x: Nat) extends Integer

defined trait Integer
defined class Positive
defined class Negative

If your definition was correct the following functions should define and print some integers



In [6]:
def one : Integer = Positive(Succ(Zero))
def neg_one : Integer = Negative(Succ(Zero))

def int_to_str(x : Integer) : String = x match {
    case Positive(x) => nat_to_str(x)
    case Negative(x) => "-" + nat_to_str(x)
}

def print_integer(x : Integer) = println(int_to_str(x))

print_integer(one)
print_integer(neg_one)

1
-1


defined function one
defined function neg_one
defined function int_to_str
defined function print_integer

## Absolute Value and Negation 

### Absolute Value (5 points)

Define the absolute value function for Integers. This should be a very simple function that takes an Integer and gives its absolute value as a $\mathbb{N}$

In [7]:
def abs(x: Integer): Nat = x match{
    case Positive(x) => x
    case Negative(x) => x
}

defined function abs

### Negation (5 points)

This should be very easy as well. Define the negation function. This makes a positive Integer negative and a negative Integer positive.

In [8]:
def neg(x: Integer): Integer = x match{
    case Positive(x) => Negative(x)
    case Negative(x) => Positive(x)
}

defined function neg

## Addition and Subtraction

Define plus and minus for Integers. Don't use the versions we created for $\mathbb{N}$ as it did some funky things to get minus to behave correctly. Try and create these from scratch instead. It is recommended to use the lte(less than or equals) function we defined last week along with abs to make the job easier. 

Note that we have renamed the operations for natural numbers so that they are of the form:



``` nat_<operation name> ```

For instance, plus has been renamed to ``` nat_plus ```. This was done so we don't have namespace conflicts

### Addition (5 points)

Fill in the missing cases for Addition

In [13]:
def plus(n : Integer, m : Integer) : Integer = (n, m) match {
    case (Positive(x), Positive(y)) => Positive(nat_plus(x, y))
    case (Negative(x), Negative(y)) => Negative(nat_plus(x, y))
    case (Negative(x), Positive(y)) => nat_lte(x, y) match {
        case True  => Positive(nat_minus(y, x))
        case False => Negative(nat_minus(x, y))
    }
    case (Positive(x), Negative(y)) => nat_lte(y,x) match {
        case True => Positive(nat_minus(x,y))
        case False => Negative(nat_minus(y,x))
    }
}

defined function plus

In [14]:
assert(plus(Positive(three), Negative(six)) == Negative(three))
assert(plus(Positive(three), Negative(two)) == Positive(Succ(Zero)))
assert(plus(Positive(five), Positive(three)) == Positive(eight))

### Subtraction (5 points)

Hint: Subtraction is very easy if you use ```plus``` and ```negate```

In [16]:
def minus(n: Integer,m: Integer) : Integer = (n,m) match {
    case _ => plus(n,neg(m))
}

defined function minus

In [17]:
assert(minus(Positive(three), Negative(six)) == Positive(nine))
assert(minus(Positive(three), Negative(two)) == Positive(five))
assert(minus(Positive(five), Positive(six)) == Negative(Succ(Zero)))

## Multiplication and Exponentiation

Write multiplication and exponentiation for Integers. You should be able to use ```nat_mult``` to greatly simplify this

### Multiplication (5 points)

Remember the rules for when to switch signs(+/-) for multiplication

In [18]:
def mult(n: Integer,m: Integer) : Integer = (n,m) match{
    case (Positive(x), Positive(y)) => Positive(nat_mult(x,y))
    case (Negative(x), Negative(y)) => Positive(nat_mult(x,y))
    case (Negative(x), Positive(y)) => Negative(nat_mult(x,y))
    case (Positive(x), Negative(y)) => Negative(nat_mult(x,y))
}

defined function mult

In [19]:
assert(mult(Positive(three), Negative(two)) == Negative(six))
assert(mult(Positive(two), Positive(Succ(Zero))) == Positive(two))
assert(mult(Negative(three), Negative(three)) == Positive(nine))

### Exponentiation (5 points)

recall that for ``pow`` we will restrict ourselves to only positive powers. Use the definition of `mult` from above so that your polarity(Positive/Negative) is correct. Recall the cases for $-x^n$ for even vs odd $n$. A hint for your base case: `pow(x, 0) = Positive( 1 )`

In [21]:
def pow(n: Integer,m: Nat) : Integer = (n,m) match{
    case (_,Zero) => Positive(Succ(Zero))
    case (_,Succ(Zero)) => n
    case (_,Succ(y)) => mult(n,pow(n,y))
}

defined function pow

In [22]:
assert(pow(Negative(two), two) == Positive(four))
assert(pow(Positive(three), Succ(Zero)) == Positive(three))
assert(pow(Negative(two), three) == Negative(eight))

# The Arithmetic Language: Our First Interpreter

Now we are ready to define our first interpreter. We will define the Arithmetic language syntax below as a sealed trait. It will be your job to correctly construct the interpreter for it based on the inference rules we covered in class. Recall that each rule corresponds to a case in the `eval` function.


In [23]:
sealed trait Expr 
case class Num(x : Integer) extends Expr 
case class Plus(x : Expr, y : Expr) extends Expr
case class Minus(x : Expr, y : Expr) extends Expr
case class Mult(x : Expr, y : Expr) extends Expr
case class Pow(x : Expr, y : Nat) extends Expr

defined trait Expr
defined class Num
defined class Plus
defined class Minus
defined class Mult
defined class Pow

## The Interpreter (15 points)

Now that we have defined the syntax for Arithmetic expressions. Go ahead and define the interpreter. We have given the signature for the function. (Bonus points if you define and use the helper function `eval-bin` that we discussed in class.

Recall that the type of this interpreter should be $eval : \text{Expr} \rightarrow \mathbb{Z}$

In [28]:
def eval (x: Expr) : Integer = x match{
    case Num(x) => x
    case Plus(x,y) => plus(eval(x),eval(y))
    case Minus(x,y) => minus(eval(x),eval(y))
    case Mult(x,y) => mult(eval(x),eval(y))
    case Pow(x,y) => pow(eval(x),y)

}

defined function eval

In [29]:
val x: Expr = Num(Positive(six))
assert(eval(x) == Positive(six))

x: Expr = Num(Positive(Succ(Succ(Succ(Succ(Succ(Succ(Zero))))))))

In [30]:
val x2: Expr = Plus(Num(Positive(two)), Num(Positive(two)))
assert(eval(x2) == Positive(four))

x2: Expr = Plus(
  Num(Positive(Succ(Succ(Zero)))),
  Num(Positive(Succ(Succ(Zero))))
)

In [31]:
val x3: Expr = Mult(Plus(Num(Positive(two)), Num(Positive(two))), Num(Negative(two)))
assert(eval(x3) == Negative(eight))

x3: Expr = Mult(
  Plus(Num(Positive(Succ(Succ(Zero)))), Num(Positive(Succ(Succ(Zero))))),
  Num(Negative(Succ(Succ(Zero))))
)